# Economic analysis of the US

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **data analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code from [lecture 5](https://numeconcopenhagen.netlify.com/lectures/Workflow_and_debugging).
> 1. Remember this [guide](https://www.markdownguide.org/basic-syntax/) on markdown and (a bit of) latex.
> 1. Turn on automatic numbering by clicking on the small icon on top of the table of contents in the left sidebar.
> 1. The `dataproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [1]:
# A wrapper for multiple APIs with a pandas interface
!pip install pandas-datareader

In [105]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
plt.style.use('seaborn-whitegrid')
from datetime import datetime

import pandas_datareader as pdr

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2
%matplotlib inline

# user written modules
#import dataproject


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


We wish to investigate the interest rate trend across the US and how it is related to developments in the unemployment rate as well as growth in GDP. The topic is of relevance given the recent years of turbulence, notably caused by the pandemic.  

# Reading and cleaning the data

First we set the date and time so that data is collected from the beginning of 2005 until the latest data point.

In [96]:
start = datetime(2005,1,1)
end = datetime.now()

For this analysis, we are going to use data for the unemployment rate, GDP, interest rate (the Federal Funds rate) and the inflation rate in the US. Each dataset is imported seperately through the FRED's API and then later we will merge all datasets into one. 

We first create a function that is used to retrieve all four datasets. By creating this function, we are able to reset the index and get the DATE field as a column, which we will need for the visualization and analysis.

In [117]:
#Function to retrieve data

def get_fred_data(start, end, var_name):
    table = pdr.DataReader(var_name, 'fred', start, end)
    return table.reset_index()

We start by importing monthly data on the US unemployment rate. Then we rename some columns and show the results in a table.

In [119]:
# Unemployment rate
series1 = 'UNRATE'

unemp = get_fred_data(start, end, var_name=[series1])
unemp.rename(columns = {'UNRATE': 'Unemp'}, inplace = True)

unemp


,DATE,Unemp
0,2005-01-01,5.3
1,2005-02-01,5.4
2,2005-03-01,5.2
3,2005-04-01,5.2
4,2005-05-01,5.1
...,...,...
202,2021-11-01,4.2
203,2021-12-01,3.9
204,2022-01-01,4.0
205,2022-02-01,3.8


Next, we collect quarterly data on US GDP. Using this variable, we create a new variable that shows the year-over-year percentage growth rate in order to get a better idea of the development of GDP.

The variable GDP is in billions of chained 2012 dollars with a seasonally adjusted annual rate. 

In [120]:
# Real Gross Domestic Product  
series2 = 'GDPC1'
gdp = get_fred_data(start, end, var_name=[series2])

gdp['pct_yoy'] = gdp['GDPC1'].pct_change(4) * 100
gdp.rename(columns = {'GDPC1': 'GDP', 'pct_yoy': 'GDP YoY growth'}, inplace = True)

gdp

,DATE,GDP,GDP YoY growth
0,2005-01-01,14767.846,NaN
1,2005-04-01,14839.707,NaN
2,2005-07-01,14956.291,NaN
3,2005-10-01,15041.232,NaN
4,2006-01-01,15244.088,3.224858
...,...,...,...
63,2020-10-01,18767.778,-2.262915
64,2021-01-01,19055.655,0.546977
65,2021-04-01,19368.310,12.226677
66,2021-07-01,19478.893,4.946556


Now we will import the daily inflation rate and rename columns.

In [121]:
# Sticky Price Consumer Price Index less Food and Energy
series3 = 'CORESTICKM159SFRBATL'
infl = get_fred_data(start, end, var_name=[series3])

infl.rename(columns={'CORESTICKM159SFRBATL': 'Inflation'}, inplace = True)

infl

,DATE,Inflation
0,2005-01-01,2.315890
1,2005-02-01,2.286245
2,2005-03-01,2.246153
3,2005-04-01,2.239722
4,2005-05-01,2.357955
...,...,...
201,2021-10-01,3.042940
202,2021-11-01,3.205069
203,2021-12-01,3.467011
204,2022-01-01,3.982988


And lastly, we import the daily Federal Funds Effective Rate and rename columns.

In [123]:
# Federal Funds Effective Rate
series4 = 'DFF'
intrate = get_fred_data(start, end, var_name=[series4])

intrate.rename(columns={'DFF': 'Int. Rate'}, inplace = True)

intrate

,DATE,Int. Rate
0,2005-01-01,1.97
1,2005-01-02,1.97
2,2005-01-03,2.31
3,2005-01-04,2.25
4,2005-01-05,2.25
...,...,...
6301,2022-04-03,0.33
6302,2022-04-04,0.33
6303,2022-04-05,0.33
6304,2022-04-06,0.33


## Explore each data set

In this section we will have a closer look at the four datasets by visualizing them in interactive plots.

**Unemployment rate** :

In [124]:
def plot_func():
    x_gdp = gdp['DATE']
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(gdp.iloc[:, 1])
    ax.set_xlabel('Year')
    ax.set_ylabel('GDP YoY Growth')
    pass;
    #x_gdp = gdp.iloc[:, 1]
    #y_gdp = timespan
    #plt.plot(x_gdp, y_gdp)

#c_slide = widgets.FloatSlider(value = 2005, min = 2005, max = 2022, step = 1, description = 'Year')



widgets.interact(plot_func, 
    c_slide = widgets.FloatSlider(value = 2005, min = 2005, max = 2022, step = 1, description = 'Year')
    )
    # Let the widget interact with data through plot_func()    



interactive(children=(Output(),), _dom_classes=('widget-interact',))

<function __main__.plot_func()>

Explain what you see when moving elements of the interactive plot around. 

# Merge data sets

Now we have fetched and explored the data, so it is time to merge all the datasets into one.

Since we have both quarterly, monthly and daily dataset, we will merge them in three steps using left joins. By doing so, the monthly and daily datasets will automatically be converted to quarterly data. We note here that using this method does not convert an average of i.e. three months to get an estimate for quarterly data. Instead, the value that will be presented in the new table, is the simply the observation of the day of each quarter, that is data observed on 20xx-01-01, 20xx-04-01, 20xx-07-01 and 20xx-10-01. All other datapoints are dropped when using the left join. This is obtained by always using the quarterly dataset as the left table. 

We will start by merging the monthly dataset on unemployment with the quarterly GDP dataset. Since both tables have a column called 'DATE' with the same format, we will merge the two tables on this column.

In [126]:
merge1 = gdp.merge(unemp, left_on = 'DATE', right_on = 'DATE')
merge1

,DATE,GDP,GDP YoY growth,Unemp
0,2005-01-01,14767.846,NaN,5.3
1,2005-04-01,14839.707,NaN,5.2
2,2005-07-01,14956.291,NaN,5.0
3,2005-10-01,15041.232,NaN,5.0
4,2006-01-01,15244.088,3.224858,4.7
...,...,...,...,...
63,2020-10-01,18767.778,-2.262915,6.9
64,2021-01-01,19055.655,0.546977,6.4
65,2021-04-01,19368.310,12.226677,6.0
66,2021-07-01,19478.893,4.946556,5.4


Next we join the new table on the left with the inflation table on the right. 

In [127]:
merge2 = merge1.merge(infl, left_on = 'DATE', right_on = 'DATE')
merge2

,DATE,GDP,GDP YoY growth,Unemp,Inflation
0,2005-01-01,14767.846,NaN,5.3,2.315890
1,2005-04-01,14839.707,NaN,5.2,2.239722
2,2005-07-01,14956.291,NaN,5.0,2.301079
3,2005-10-01,15041.232,NaN,5.0,2.325302
4,2006-01-01,15244.088,3.224858,4.7,2.451019
...,...,...,...,...,...
63,2020-10-01,18767.778,-2.262915,6.9,1.848308
64,2021-01-01,19055.655,0.546977,6.4,1.504532
65,2021-04-01,19368.310,12.226677,6.0,2.284027
66,2021-07-01,19478.893,4.946556,5.4,2.325734


Lastly, this new table is then merged with the interest rate table. Again, we use the merge2 table on the left, the interest rate table on the right and we merge on the 'DATE' column. 

Since the GDP YoY growth has no values for 2005, we drop those rows so the final table only includes rows with no missing values.

In [128]:
final_table = merge2.merge(intrate, left_on = 'DATE', right_on = 'DATE')
final_table.dropna(inplace = True)
final_table

,DATE,GDP,GDP YoY growth,Unemp,Inflation,Int. Rate
4,2006-01-01,15244.088,3.224858,4.7,2.451019,4.09
5,2006-04-01,15281.525,2.977269,4.7,2.680693,5.00
6,2006-07-01,15304.517,2.328291,4.7,3.093999,5.05
7,2006-10-01,15433.643,2.608902,4.4,3.291125,5.34
8,2007-01-01,15478.956,1.540715,4.6,3.145734,5.17
...,...,...,...,...,...,...
63,2020-10-01,18767.778,-2.262915,6.9,1.848308,0.09
64,2021-01-01,19055.655,0.546977,6.4,1.504532,0.09
65,2021-04-01,19368.310,12.226677,6.0,2.284027,0.07
66,2021-07-01,19478.893,4.946556,5.4,2.325734,0.10


We now have one table, called final_table, with observations on GDP, GDP YoY growth, unemployment, inflation and interest rate for each quarter from 2006 until the last observed quarter, which is 2021-10-01. 

# Analysis

We would like to take a closer look at the development of the variables during the past few years to better see how the Covid-19 pandemic has affected the economy in the US. We can do this by using indexing. However, since we don't want to overide this indexing with the full table, we create a copy.

In [129]:
final_table_copy = final_table.copy()
final_table_copy.loc[final_table_copy.DATE > '2017-01-01']

,DATE,GDP,GDP YoY growth,Unemp,Inflation,Int. Rate
49,2017-04-01,17996.802,2.146717,4.4,2.252170,0.82
50,2017-07-01,18126.226,2.266565,4.3,2.146769,1.06
51,2017-10-01,18296.685,2.717886,4.2,2.192819,1.06
52,2018-01-01,18436.262,3.015312,4.0,2.166829,1.33
53,2018-04-01,18590.004,3.296152,4.0,2.492147,1.67
54,2018-07-01,18679.599,3.052886,3.8,2.531541,1.91
55,2018-10-01,18721.281,2.320617,3.8,2.419846,2.18
56,2019-01-01,18833.195,2.153002,4.0,2.378344,2.40
57,2019-04-01,18982.528,2.111479,3.6,2.410959,2.41
58,2019-07-01,19112.653,2.318326,3.7,2.467932,2.39


MAKE FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

# Conclusion

ADD CONCISE CONLUSION.